In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 9 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Challenge - dynamique de la chaleur
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Challenge - dynamique de la chaleur

In [ ]:
from dataclasses import dataclass
import numpy as np
from scipy.integrate import solve_ivp
from scipy.sparse import diags
from scipy import sparse
from scipy.sparse.linalg import spsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

On considère l'équation de la chaleur dans le cas 1D :

$$
\left\{
\begin{aligned}
& \partial_t u(x,t) - D \, \partial_{xx} u(x,t) = 0 \;\; \text{pour} \; 0 < x < 1\\
& \partial_{x} u(0,t) = \partial_{x} u(1,t) = 0,\\
& u(x,t=t_0)  =  u_0(x),
\end{aligned}
\right.
$$

où la donnée initiale est choisie par l'utilisateur et avec des conditions aux limites de type Neumann homogène.

Pour la simulation nous prenons deux temps finaux  $t_0=0.005$ et $t_0=0.015$ et $D=1$. Nous prenons aussi 1001 points de maillage en espace et 10001 points en temps.

In [ ]:
#############################################
class heat_model:

    def __init__(self, d, xmin, xmax, nx) :
        self.d = d
        self.xmin = xmin
        self.xmax = xmax
        self.nx = nx
        self.dx = (xmax-xmin)/(nx-1)
        self.oneoverdxdx = 1/(self.dx*self.dx)
        
        doverdxdx = d/(self.dx**2)
        diag = np.repeat(-2, nx)
        sub_diag = np.repeat([1., 2.], [nx-2, 1])
        self.a = doverdxdx * diags([diag, sub_diag, np.flip(sub_diag)], [0, -1, 1])

    def fcn(self, t, y):
        return self.a.dot(y)
    
    def fcn_exact(self, t):
        xmin = self.xmin
        xmax = self.xmax
        d = self.d
        nx = self.nx
        dx = self.dx
        x = np.linspace(xmin, xmax, nx)
        y = (1/(2*np.sqrt(np.pi*t))) * np.exp(-(x*x)/(4.*t))
        return y

## Méthode d'Euler explicite

In [ ]:
@dataclass
class ode_result:
    t: np.ndarray 
    y: np.ndarray    
    
#############################################
def forward_euler(tini, tend, nt, yini, fcn):
    
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    for it, tn  in enumerate(t[:-1]):
        yn = yn + dt*fcn(tn, yn)
        
    return ode_result(t, yn)

## Dynamique de l'équation de la chaleur 

Nous considérons une donnée initiale à 2 modes 

$u_0(x) = \cos\Big(2\pi x\Big) + 2 \cos\Big(6\pi x\Big)$


In [ ]:
xmin = 0.
xmax = 1.
nx = 1001
dx = (xmax-xmin)/(nx-1)
x = np.linspace(xmin, xmax, nx)

d = 1.    
hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nx)
fcn = hm.fcn

tini = 0.
tend = 0.005

k = 2*np.pi

yini = np.cos(k*x) + 2*np.cos(3*k*x)

yexa = np.cos(k*x) * np.exp(-k**2 * tend) + 2*np.cos(3*k*x) * np.exp(-(3*k)**2 * tend)
yexa3 = np.cos(k*x) * np.exp(-k**2 * 3* tend) + 2*np.cos(3*k*x) * np.exp(-(3*k)**2 * 3 * tend)

nt = 10001
dt = (tend-tini) / (nt-1)
print(f"dt = {dt}")

sol = forward_euler(tini, tend, nt, yini, fcn)

solsol = forward_euler(tini, 3*tend, 3*(nt-1)+1, yini, fcn)

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=x, y=yini, name="sol. ini."))
fig_sol.add_trace(go.Scatter(x=x, y=yexa, name="sol. exa."))
fig_sol.add_trace(go.Scatter(x=x, y=sol.y, name="sol. num. tend = 0.005"))
fig_sol.add_trace(go.Scatter(x=x, y=solsol.y, name="sol. num. tend = 0.015"))
legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)')
fig_sol.update_layout(legend=legend, height=450)
fig_sol.update_xaxes(title='x')
fig_sol.show()

fig_err = go.Figure()
fig_err.add_trace(go.Scatter(x=x, y=sol.y-yexa, name="err - tend = 0.005", showlegend=True))
fig_err.add_trace(go.Scatter(x=x, y=solsol.y-yexa3, name="err - tend = 0.015", showlegend=True))
fig_err.update_layout(legend=legend, height=450)
fig_err.update_xaxes(title='x')
fig_err.update_yaxes(title='ynum - yexa', exponentformat='e')
fig_err.show()

Interpréter la dynamique de manière quantitative ? Que reste-t-il de la fréquence la plus élevée au temps $t=0.015$ ? Expliquer.